    # 0. Установка и Импорт

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
!pip install yt-dlp

In [8]:
from googleapiclient.discovery import build
import requests
import pickle
import yt_dlp
import json
import re

In [5]:
API_KEY = 'your_api_key'
CHANNEL_ID = 'UCdxesVp6Fs7wLpnp1XKkvZg'

In [6]:
youtube = build('youtube', 'v3', developerKey=API_KEY)

# 1. Скачиваем список и описание всех видео

In [ ]:
def get_channel_videos(channel_id):
    """
    Функция, которая принимает на вход id youtube канала, а возвращает
    массив id всех его видео
    """
    video_ids = []
    response = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    ).execute()

    uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    next_page_token = None
    while True:
        playlist_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = playlist_response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

def get_video_details(video_ids):
    """
    Функция, которая принимает на вход массив id видео, а возвращает
    массив с мета информацией о этих видео, включая:
      1. Id (video_id)
      2. Название (title)
      3. Описание (description)
      4. Дата публикации (published_at)
      5. Название канала (channel_title)
      6. Теги (tags)
      7. Id категории (category_id)
      8. Количество просмотров (view_count)
      9. Количество лайков (like_count)
      10. Количество комментариев (comment_count)
    """
    videos = []
    for i in range(0, len(video_ids), 50):  # API позволяет запрашивать до 50 видео за раз
        response = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids[i:i+50])
        ).execute()

        for item in response['items']:
            video_info = {
                'video_id': item['id'],
                'title': item['snippet']['title'],
                'description': item['snippet']['description'],
                'published_at': item['snippet']['publishedAt'],
                'channel_title': item['snippet']['channelTitle'],
                'tags': item['snippet'].get('tags', []),
                'category_id': item['snippet']['categoryId'],
                'view_count': item['statistics'].get('viewCount', 0),
                'like_count': item['statistics'].get('likeCount', 0),
                'comment_count': item['statistics'].get('commentCount', 0)
            }
            videos.append(video_info)
    return videos

video_ids = get_channel_videos(CHANNEL_ID)
video_details = get_video_details(video_ids)


# Сохраняем исходные выгруженные сырые данные
with open('video_details_raw.json', 'w', encoding='utf-8') as fp:
    json.dump(video_details, fp, ensure_ascii=False, indent=4)


with open('video_ids.pickle', 'wb') as fp:
    pickle.dump(video_ids, fp)

In [ ]:
print(f"video_ids (LEN = {len(video_ids)}):\n[{video_ids[0]}, {video_ids[1]}, {video_ids[2]}, ...]\n")
print("-"*20)
print(F"video_details (LEN = {len(video_details)}):\n")
for key in video_details[0]:
    print(f"{key}: {video_details[0][key]}")

video_ids (LEN = 4094):
[sDwkFzbDZWQ, NjlDypgNp3s, JXnnlhm4Z0c, ...]

--------------------
video_details (LEN = 4094):

video_id: sDwkFzbDZWQ
title: Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики
description: Таймкоды:
0:00 Начало
3:29 Доказательство утв-я про классы эквивалентности
13:49 Построение МПДКА по регулярке
14:29 Конечность классов эквивалентности
17:41 Теорема Майхилла-Нероуда
19:03 Пример доказательства неавтоматности языка
23:04 Порождающие грамматики
26:11 Пример грамматики
32:36 Выводимость в грамматике
37:22 Иерархия Хомского
47:34 Теорема об эквивалентности праволинейных грамматик и автоматных языков

Лектор: Ахтямов Павел Ибрагимович
Дата лекции: 01.10.2024

Оператор: Савельев Е.
Монтажер: Андреев Я.

Плейлист: https://www.youtube.com/playlist?list=PL4_hYwCyhAvaB9lc47qlPq_XteD3l7HIs
published_at: 2024-10-31T14:47:05Z
channel_title: Лекторий ФПМИ
tags: []
category_id: 27
view_count: 67
like_count: 1
comment_count: 0


# 2. Предобрабатываем данные

В заголовке можно вычленить отдельно тему и название лекции

In [ ]:
def pars_title(title):
    """
    Функция, которая вычленяет из заголовка тему и само название лекции
    """
    title_split = title.split(".")
    topic = title_split[0].strip("123456789 ")
    lecture_title = '.'.join(title_split[1:]).strip(' ')
    return topic, lecture_title

In [ ]:
VIDEO_COUNT = 3
for ind, video in enumerate(video_details[:VIDEO_COUNT]):
    print(f"------\nВидео {ind + 1}.")
    print(video['title'])
    topic, lecture_title = pars_title(video['title'])
    print(f"\nТема: {topic}\nНазвание видео: {lecture_title}\n------\n")

------
Видео 1.
Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики

Тема: Формальные языки и трансляции
Название видео: Построение минимального автомата. Праволинейные грамматики
------

------
Видео 2.
Формальные языки и трансляции 4. МПДКА. Праволинейные грамматики

Тема: Формальные языки и трансляции
Название видео: МПДКА. Праволинейные грамматики
------

------
Видео 3.
Введение в математический анализ 17. Дифференциал. Теоремы о среднем.

Тема: Введение в математический анализ
Название видео: Дифференциал. Теоремы о среднем.
------



In [ ]:
for ind, video in enumerate(video_details):
    topic, lecture_title = pars_title(video['title'])
    video_details[ind]['Tема'] = topic
    video_details[ind]['Название лекции'] = lecture_title

Описание видео можно разбить на несколько частей (каждая их кооторых может и не присутсвовать в данном описании)


1. Таймкоды
2. Лектор
3. Оператор
4. Монтажер
5. Плейлист

In [ ]:
timestamps_pattern = re.compile(r"(\d{1,2}:\d{2})\s+(.+)")
lector_pattern = re.compile(r"Лектор:\s+(.+)")
operator_pattern = re.compile(r"Оператор:\s+(.+)")
editor_pattern = re.compile(r"Монтаж[ёе]р:\s+(.+)")
playlist_pattern = re.compile(r"Плейлист:\s+(https?://[^\s]+)")
date_pattern = re.compile(r"Дата лекции:\s+(\d{2}\.\d{2}\.\d{4})")

def pars_description(text):
      """
      Функция, которая вычленяет из описания видео:
        1. Таймкоды
        2. Лектора
        3. Оператора
        4. Монтажера
        5. Ссылку на плейлист
        6. Дату лекции
      """
      timestamps = timestamps_pattern.findall(text)
      timestamps = [(timestamp[0], timestamp[1].strip(' -,')) for timestamp in timestamps]
      lector = lector_pattern.search(text).group(1) if lector_pattern.search(text) else None
      operator = operator_pattern.search(text).group(1) if operator_pattern.search(text) else None
      editor = editor_pattern.search(text).group(1) if editor_pattern.search(text) else None
      playlist = playlist_pattern.search(text).group(1) if playlist_pattern.search(text) else None
      date = date_pattern.search(text).group(1) if date_pattern.search(text) else None

      return timestamps, lector, operator, editor, playlist, date


for ind, video in enumerate(video_details):
    timestamps, lector, operator, editor, playlist, date = pars_description(video['description'])
    video_details[ind]["Таймкоды"] = timestamps
    video_details[ind]["Лектор"] = lector
    video_details[ind]["Оператор"] = operator
    video_details[ind]["Монтажер"] = editor
    video_details[ind]["Плейлист"] = playlist
    video_details[ind]["Дата лекции"] = date

In [ ]:
VIDEO_COUNT_2 = 2
for ind, video in enumerate(video_details[:VIDEO_COUNT_2]):
    print(f"------\nВидео {ind + 1}.")
    for key in video:
        print(f"{key}: {video[key]}")
    print()

------
Видео 1.
video_id: sDwkFzbDZWQ
title: Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики
description: Таймкоды:
0:00 Начало
3:29 Доказательство утв-я про классы эквивалентности
13:49 Построение МПДКА по регулярке
14:29 Конечность классов эквивалентности
17:41 Теорема Майхилла-Нероуда
19:03 Пример доказательства неавтоматности языка
23:04 Порождающие грамматики
26:11 Пример грамматики
32:36 Выводимость в грамматике
37:22 Иерархия Хомского
47:34 Теорема об эквивалентности праволинейных грамматик и автоматных языков

Лектор: Ахтямов Павел Ибрагимович
Дата лекции: 01.10.2024

Оператор: Савельев Е.
Монтажер: Андреев Я.

Плейлист: https://www.youtube.com/playlist?list=PL4_hYwCyhAvaB9lc47qlPq_XteD3l7HIs
published_at: 2024-10-31T14:47:05Z
channel_title: Лекторий ФПМИ
tags: []
category_id: 27
view_count: 67
like_count: 1
comment_count: 0
Tема: Формальные языки и трансляции
Название лекции: Построение минимального автомата. Праволинейные грамматики

In [ ]:
# Сохраянем
with open('video_details_preprocessing.json', 'w', encoding='utf-8') as fp:
    json.dump(video_details, fp, ensure_ascii=False, indent=4)

# 3. Скачиваем видео и отделяем от них аудио

In [11]:
import yt_dlp

def download_youtube_audio(url: str, output_path: str = "."):
    """
    Скачивает аудио с YouTube по переданному URL и сохраняет его в формате MP3 с помощью yt-dlp.

    :param url: Ссылка на видео с YouTube.
    :param output_path: Путь, по которому будет сохранено аудио.
    :return: Путь к скачанному файлу.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        audio_title = info_dict.get('title', 'audio')
        downloaded_file_path = f"{output_path}/{audio_title}.mp3"

    return downloaded_file_path


url = "https://www.youtube.com/watch?v=sDwkFzbDZWQ"
path_to_audio = download_youtube_audio(url, output_path=".")
print(f"Аудио скачано по пути: {path_to_audio}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=sDwkFzbDZWQ
[youtube] sDwkFzbDZWQ: Downloading webpage
[youtube] sDwkFzbDZWQ: Downloading ios player API JSON
[youtube] sDwkFzbDZWQ: Downloading mweb player API JSON
[youtube] sDwkFzbDZWQ: Downloading m3u8 information
[info] sDwkFzbDZWQ: Downloading 1 format(s): 251
[download] Destination: ./Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики.webm
[download] 100% of   57.74MiB in 00:00:01 at 37.72MiB/s  
[ExtractAudio] Destination: ./Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики.mp3
Deleting original file ./Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики.webm (pass -k to keep)
Аудио скачано по пути: ./Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики.mp3
